In [1]:
import numpy as np
import cv2
import tensorflow as tf
import os
import glob
# import pathlib
# from sklearn.metrics import classification_report, accuracy_score
from matplotlib import pyplot as plt
%matplotlib inline

In [2]:
# Загрузим изображения отдельно для обучения и валидации из папок train и test, и приведем к одинаковому размеру
img_size = [224, 224]

train_data = []
train_labels = []

val_data = []
val_labels = []

training_paths = glob.glob('../input/hymenoptera-data/hymenoptera_data/train/ants/*.jpg') +\
                 glob.glob('../input/hymenoptera-data/hymenoptera_data/train/bees/*.jpg')

val_paths = glob.glob('../input/hymenoptera-data/hymenoptera_data/val/ants/*.jpg') +\
                 glob.glob('../input/hymenoptera-data/hymenoptera_data/val/bees/*.jpg')

for image_path in training_paths:
    img =cv2.imread(image_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, (img_size[0], img_size[1])).flatten()
    img = np.resize(img, (img_size[0], img_size[1], 3))
    train_data.append(img)
    
    lbl = image_path.split(os.path.sep)[-2]
    train_labels.append(lbl)

for image_path in val_paths:
    img =cv2.imread(image_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, (img_size[0], img_size[1])).flatten()
    img = np.resize(img, (img_size[0], img_size[1], 3))
    val_data.append(img)
    
    lbl = image_path.split(os.path.sep)[-2]
    val_labels.append(lbl)

print(f'Train length:      {len(train_data)},    train data shape: {train_data[0].shape}')
print(f'Validation length: {len(val_data)}, validation data shape: {val_data[0].shape}')

Train length:      243,    train data shape: (224, 224, 3)
Validation length: 153, validation data shape: (224, 224, 3)


In [3]:
# Приведем к типу numpy.ndarray
train_data = np.array(train_data)
val_data = np.array(val_data)
# Заменим метки числами и приведем к типу numpy.ndarray
train_labels = np.array([0 if x=='ants' else 1 for x in train_labels])
val_labels = np.array([0 if x=='ants' else 1 for x in val_labels])

---

Обучим нейронную сеть с использованием свёрточных, пулинговых и полносвязных слоёв для бинарной классификации.

In [4]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(filters=6,
                           kernel_size=(5, 5),
                           padding='same',
                           activation='relu',
                           input_shape=(img_size[0], img_size[1], 3)),
    tf.keras.layers.MaxPool2D(pool_size=(2, 2), padding='valid'),
    
    tf.keras.layers.Conv2D(filters=16, 
                           kernel_size=(5, 5),
                           padding='same',
                           activation='relu'),
    tf.keras.layers.MaxPool2D(pool_size=(2, 2), padding='valid'),
    
    tf.keras.layers.Flatten(),
#     tf.keras.layers.Dropout(.2),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')])

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [5]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 224, 224, 6)       456       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 112, 112, 6)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 112, 112, 16)      2416      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 56, 56, 16)        0         
_________________________________________________________________
flatten (Flatten)            (None, 50176)             0         
_________________________________________________________________
dense (Dense)                (None, 32)                1605664   
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 3

In [6]:
model.fit(train_data, 
          train_labels,
          batch_size=32, 
          epochs=10,
          validation_data=(val_data, val_labels))

Epoch 1/10
8/8 [==============================] - 2s 105ms/step - loss: 89.2737 - accuracy: 0.5140 - val_loss: 5.2953 - val_accuracy: 0.6013
Epoch 2/10
8/8 [==============================] - 0s 22ms/step - loss: 4.6306 - accuracy: 0.6163 - val_loss: 2.3938 - val_accuracy: 0.5098
Epoch 3/10
8/8 [==============================] - 0s 21ms/step - loss: 1.4843 - accuracy: 0.6628 - val_loss: 2.0697 - val_accuracy: 0.6078
Epoch 4/10
8/8 [==============================] - 0s 21ms/step - loss: 1.7194 - accuracy: 0.6311 - val_loss: 1.2619 - val_accuracy: 0.5163
Epoch 5/10
8/8 [==============================] - 0s 21ms/step - loss: 0.5409 - accuracy: 0.7616 - val_loss: 1.1448 - val_accuracy: 0.5359
Epoch 6/10
8/8 [==============================] - 0s 21ms/step - loss: 0.3442 - accuracy: 0.8041 - val_loss: 0.9093 - val_accuracy: 0.5948
Epoch 7/10
8/8 [==============================] - 0s 21ms/step - loss: 0.3088 - accuracy: 0.8630 - val_loss: 0.8463 - val_accuracy: 0.5752
Epoch 8/10
8/8 [=========

На тренировочных данных с каждой эпохой функция loss уменьшается, а метрика accuracy увеличивается, но на валидационных данных метрика меняется в интервале около 0.5 до 0.6.

---

Добавим в нейронную сеть  слои BatchNormalization.

In [7]:
model2 = tf.keras.models.Sequential([
    
    tf.keras.layers.Conv2D(filters=6,
                           kernel_size=(5, 5),
                           padding='same',
                           activation='relu',
                           input_shape=(img_size[0], img_size[1], 3)),
    
    tf.keras.layers.BatchNormalization(),
    
    tf.keras.layers.MaxPool2D(pool_size=(2, 2), padding='valid'),
    
    tf.keras.layers.Conv2D(filters=16, 
                           kernel_size=(5, 5),
                           padding='same',
                           activation='relu'),
    tf.keras.layers.BatchNormalization(),
    
    tf.keras.layers.MaxPool2D(pool_size=(2, 2), padding='valid'),
    
    tf.keras.layers.Flatten(),
    
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')])

model2.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [8]:
model2.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_2 (Conv2D)            (None, 224, 224, 6)       456       
_________________________________________________________________
batch_normalization (BatchNo (None, 224, 224, 6)       24        
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 112, 112, 6)       0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 112, 112, 16)      2416      
_________________________________________________________________
batch_normalization_1 (Batch (None, 112, 112, 16)      64        
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 56, 56, 16)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 50176)            

In [9]:
model2.fit(train_data, 
          train_labels,
          batch_size=32, 
          epochs=10,
          validation_data=(val_data, val_labels))

Epoch 1/10
8/8 [==============================] - 1s 68ms/step - loss: 4.2149 - accuracy: 0.5893 - val_loss: 2.9874 - val_accuracy: 0.6732
Epoch 2/10
8/8 [==============================] - 0s 30ms/step - loss: 0.8459 - accuracy: 0.6818 - val_loss: 2.5518 - val_accuracy: 0.6078
Epoch 3/10
8/8 [==============================] - 0s 27ms/step - loss: 0.4735 - accuracy: 0.6585 - val_loss: 1.0117 - val_accuracy: 0.6144
Epoch 4/10
8/8 [==============================] - 0s 33ms/step - loss: 0.4018 - accuracy: 0.7136 - val_loss: 1.1022 - val_accuracy: 0.6013
Epoch 5/10
8/8 [==============================] - 0s 28ms/step - loss: 0.3497 - accuracy: 0.7992 - val_loss: 1.1228 - val_accuracy: 0.6209
Epoch 6/10
8/8 [==============================] - 0s 30ms/step - loss: 0.2712 - accuracy: 0.8433 - val_loss: 1.1271 - val_accuracy: 0.6601
Epoch 7/10
8/8 [==============================] - 0s 26ms/step - loss: 0.2413 - accuracy: 0.8474 - val_loss: 1.3348 - val_accuracy: 0.6078
Epoch 8/10
8/8 [===========

В этом случае метрика на валидационной выборке получилась хуже, чем в первой модели. При каждом запуске значения меняются. Видимо, это связано с инициализацией весов. В каждой эпохе значения метрики на валидационной выборке меняются в интервале от 0.55 до 0.67, и среднее значение метрики больше, чем в первой модели.

---

Обучим нейронную сеть ResNet50 на нашей обучающей выборке, инициализированную случайными весами.

In [10]:
base_model = tf.keras.applications.ResNet50(
    include_top=False, weights=None, input_tensor=None,
    input_shape=(224, 224, 3), pooling=None, classes=1
)

x = base_model.output
x = tf.keras.layers.Flatten()(x)
# x = tf.keras.layers.Dropout(.2)(x)
# x = tf.keras.layers.Dense(32, activation='relu')(x)
predictions = tf.keras.layers.Dense(1, activation='sigmoid')(x)
model3 = tf.keras.Model(inputs=base_model.input, outputs=predictions)

optimizer = tf.keras.optimizers.Adam(0.0001)

model3.compile(optimizer=optimizer,
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [11]:
model3.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 112, 112, 64) 256         conv1_conv[0][0]                 
______________________________________________________________________________________________

In [12]:
model3.fit(train_data, 
          train_labels,
          batch_size=32, 
          epochs=10,
          validation_data=(val_data, val_labels))

Epoch 1/10
8/8 [==============================] - 9s 458ms/step - loss: 2.3654 - accuracy: 0.5158 - val_loss: 3.7433 - val_accuracy: 0.5425
Epoch 2/10
8/8 [==============================] - 2s 200ms/step - loss: 1.6230 - accuracy: 0.5833 - val_loss: 1.1948 - val_accuracy: 0.5490
Epoch 3/10
8/8 [==============================] - 2s 199ms/step - loss: 1.3553 - accuracy: 0.6135 - val_loss: 0.6716 - val_accuracy: 0.6078
Epoch 4/10
8/8 [==============================] - 2s 199ms/step - loss: 0.7524 - accuracy: 0.6962 - val_loss: 0.6956 - val_accuracy: 0.5817
Epoch 5/10
8/8 [==============================] - 2s 200ms/step - loss: 0.5708 - accuracy: 0.7489 - val_loss: 0.7126 - val_accuracy: 0.4641
Epoch 6/10
8/8 [==============================] - 2s 200ms/step - loss: 0.3989 - accuracy: 0.7680 - val_loss: 0.7513 - val_accuracy: 0.4510
Epoch 7/10
8/8 [==============================] - 2s 199ms/step - loss: 0.3393 - accuracy: 0.8377 - val_loss: 0.7713 - val_accuracy: 0.4575
Epoch 8/10
8/8 [====

Функция loss на тренировочной выборке с каждой эпохой сводится к минимуму, но значение метрики на валидационной выборке остановилось на значении 0.4575, т.е. точности предсказания у модели нет никакой.

---

Обучим модель ResNet50 на обучающей выборке, инициализированную весами ImageNet.

In [13]:
train_weights_lbls = np.zeros((train_data.shape[0], 1000))
for i in range(train_data.shape[0]):
    if i < train_labels.argmax():
        train_weights_lbls[i][0] = 1
    else:
        train_weights_lbls[i][1] = 1

val_weights_lbls = np.zeros((val_data.shape[0], 1000))
for i in range(val_data.shape[0]):
    if i < val_labels.argmax():
        val_weights_lbls[i][0] = 1
    else:
        val_weights_lbls[i][1] = 1

In [14]:
model4 = tf.keras.applications.ResNet50(
    include_top=True, weights='imagenet', input_tensor=None,
    input_shape=(img_size[0], img_size[1], 3), pooling=None
)

model4.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

102973440/102967424 [==============================] - 1s 0us/step


In [15]:
model4.fit(train_data, 
          train_weights_lbls,
          batch_size=32, 
          epochs=10,
          validation_data=(val_data, val_weights_lbls))

Epoch 1/10
8/8 [==============================] - 8s 348ms/step - loss: 4.6869 - accuracy: 0.4295 - val_loss: 18.7152 - val_accuracy: 0.5621
Epoch 2/10
8/8 [==============================] - 2s 202ms/step - loss: 1.0249 - accuracy: 0.7840 - val_loss: 226.2765 - val_accuracy: 0.5425
Epoch 3/10
8/8 [==============================] - 2s 201ms/step - loss: 0.4533 - accuracy: 0.8628 - val_loss: 424.1703 - val_accuracy: 0.4575
Epoch 4/10
8/8 [==============================] - 2s 201ms/step - loss: 0.2993 - accuracy: 0.8902 - val_loss: 215.0397 - val_accuracy: 0.4575
Epoch 5/10
8/8 [==============================] - 2s 201ms/step - loss: 0.1430 - accuracy: 0.9305 - val_loss: 35.2236 - val_accuracy: 0.3856
Epoch 6/10
8/8 [==============================] - 2s 201ms/step - loss: 0.0698 - accuracy: 0.9738 - val_loss: 11.1482 - val_accuracy: 0.3922
Epoch 7/10
8/8 [==============================] - 2s 200ms/step - loss: 0.0687 - accuracy: 0.9883 - val_loss: 10.0275 - val_accuracy: 0.5490
Epoch 8/10

Значение функции loss на обучающей выборке уменьшается, значение метрики увеличивается. На валидационной выборке метрика варьируется, последнее значение около 0.7.